In [2]:
import os
import streamlit as st
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from tools import tool1_weather, tool2_stock, tool3_general_qa


In [3]:

load_dotenv()

True

In [5]:
class AgentState(TypedDict, total=False):
    messages: Annotated[list, "shared"]
    chat_history: list  
    weather_data: str
    stock_data: str
    final_answer: str
    next_tools: list[str]

In [4]:
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model="llama3-70b-8192")

In [6]:
if "memory" not in st.session_state:
    st.session_state.memory = ConversationBufferMemory(return_messages=True)

memory = st.session_state.memory  

2025-07-23 13:58:37.711 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:58:37.713 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
C:\Users\Mahendra\AppData\Local\Temp\ipykernel_17728\1046608496.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  st.session_state.memory = ConversationBufferMemory(return_messages=True)
2025-07-23 13:58:37.713 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 13:58:37.713 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
20

In [7]:
import ast  

def planner_node(state: AgentState) -> AgentState:
    query = state["messages"][-1].content.lower()

    tool_prompt = f"""
You are a tool selector for a multi-agent AI assistant.

Your job is to return ONLY a Python list of tool names to use (in correct order). Do NOT explain, comment, or add anything else.

Available tools:
- "weather_agent": for weather, temperature, or forecast-related queries
- "stock_agent": for stock prices, company tickers, or market-related questions
- "qa_agent": always include this LAST for summarizing final answers

User query:
"{query}"

Respond with a Python list of tool names. Example:
["weather_agent", "stock_agent", "qa_agent"]
"""

    raw_response = llm.invoke(tool_prompt).content.strip()
    print("🧠 Tool planner raw response:", raw_response)

    # Safe parsing
    try:
        selected_tools = ast.literal_eval(raw_response)
        if not isinstance(selected_tools, list):
            raise ValueError("Not a valid list")
    except Exception as e:
        print("❌ Failed to parse tools:", e)
        # Fallback to QA only (if LLM messes up)
        selected_tools = ["qa_agent"]

    # Ensure QA agent is last
    if "qa_agent" not in selected_tools:
        selected_tools.append("qa_agent")

    print("✅ Final selected tools:", selected_tools)
    return {**state, "next_tools": selected_tools}


In [8]:
def router_fn(state: AgentState) -> str:
    tools = state.get("next_tools", [])
    if tools:
        return tools[0]  
    return "qa_agent" 

In [9]:

weather_agent = RunnableLambda(lambda state: {
    **state,
    "weather_data": tool1_weather.invoke(state["messages"][-1].content),
    "next_tools": state["next_tools"][1:],
})


In [10]:
stock_agent = RunnableLambda(lambda state: {
    **state,
    "stock_data": tool2_stock.invoke(state["messages"][-1].content),
    "next_tools": state["next_tools"][1:],
})

In [11]:

from langchain_core.messages import HumanMessage, AIMessage

def qa_agent_node(state: AgentState) -> AgentState:
    user_input = state["messages"][-1].content.lower()
    past_messages = state.get("chat_history", [])

   
    if "summarize" in user_input or "summary" in user_input:
        filtered = [msg for msg in past_messages if "summarize" not in msg.content.lower()]
        num_user = sum(isinstance(msg, HumanMessage) for msg in filtered)
        num_bot = sum(isinstance(msg, AIMessage) for msg in filtered)

        if num_user < 1 or num_bot < 1:
            summary = "There is not enough conversation to summarize yet."
        else:
            history_text = "\n".join(
                f"{'User' if isinstance(msg, HumanMessage) else 'Bot'}: {msg.content}"
                for msg in filtered
            )
            # prompt = (
            #     "You are a helpful assistant. Summarize the following conversation:\n\n"
            #     f"{history_text}\n\nWrite a clear and concise summary."
            # )
            # summary = llm.invoke(prompt).content.strip()
            
            prompt = (
                f"You are a helpful assistant. The user asked: \"{user_input}\"\n\n"
                f"{context.strip()}\n\n"
                f"⚠️ IMPORTANT: Do not reformat or alter the weather forecast output. "
                f"Keep the exact structure and formatting as shown — especially dates like 'Wednesday, 24 July 2025'. "
                f"Do not replace these with 'Day 1', 'Day 2', etc. Just return the full response exactly."
            )


            response = llm.invoke(prompt)

        updated_history = past_messages + [AIMessage(content=summary)]
        return {**state, "final_answer": summary, "chat_history": updated_history, "next_tools": []}

    
    context = ""
    if state.get("weather_data"):
        context += f"\n**Weather Info:**\n{state['weather_data']}\n"
    if state.get("stock_data"):
        context += f"\n**Stock Info:**\n{state['stock_data']}\n"

    prompt = (
        f"You are a helpful assistant. The user asked: \"{user_input}\"\n\n"
        f"{context.strip()}\n\nKeep it concise."
    )
    response = llm.invoke(prompt)
    updated_history = past_messages + [HumanMessage(content=user_input), AIMessage(content=response.content)]

    return {
        **state,
        "final_answer": response.content,
        "chat_history": updated_history,
        "next_tools": []
    }


In [12]:

graph = StateGraph(AgentState)
graph.set_entry_point("planner")

graph.add_node("planner", planner_node)
graph.add_node("weather_agent", weather_agent)
graph.add_node("stock_agent", stock_agent)
graph.add_node("qa_agent", qa_agent_node)

graph.add_conditional_edges("planner", router_fn, {
    "weather_agent": "weather_agent",
    "stock_agent": "stock_agent",
    "qa_agent": "qa_agent",
})
graph.add_conditional_edges("weather_agent", router_fn, {
    "weather_agent": "weather_agent",
    "stock_agent": "stock_agent",
    "qa_agent": "qa_agent",
})
graph.add_conditional_edges("stock_agent", router_fn, {
    "weather_agent": "weather_agent",
    "stock_agent": "stock_agent",
    "qa_agent": "qa_agent",
})
graph.add_edge("qa_agent", END)

multiagent_app = graph.compile()

In [13]:

def invoke_multiagent(user_input: str) -> str:
    # ✅ Add user input to memory
    memory.chat_memory.add_user_message(user_input)

    # ✅ Fetch full message history from memory
    full_messages = memory.chat_memory.messages

    # ✅ Create LangGraph-compatible state with full history
    initial_state = {
        "messages": full_messages,
        "chat_history": full_messages 
    }

    # ✅ Run LangGraph with full memory-aware context
    result = multiagent_app.invoke(initial_state)

    # ✅ Add final assistant reply to memory
    memory.chat_memory.add_ai_message(result["final_answer"])

    return result["final_answer"]

In [14]:

if __name__ == "__main__":
    try:
        import streamlit.web.bootstrap
        IS_STREAMLIT = True
    except ImportError:
        IS_STREAMLIT = False

    if IS_STREAMLIT or "STREAMLIT_SERVER_HEADLESS" in os.environ:
        st.set_page_config(page_title="LangGraph Chatbot", layout="centered")
        st.title("🧠 LangGraph Multi-Agent Chatbot")
        st.markdown("Ask about weather, stocks, or general questions!")

        if "chat_history" not in st.session_state:
            st.session_state.chat_history = []

        user_input = st.chat_input("Type your message...")

        if user_input:
            st.session_state.chat_history.append(("user", user_input))
            with st.spinner("🤔 Thinking..."):
                reply = invoke_multiagent(user_input)
            st.session_state.chat_history.append(("bot", reply))

        for role, message in st.session_state.chat_history:
            with st.chat_message(role):
                st.markdown(message)
    else:
        while True:
            user_input = input("You: ")
            if user_input.strip().lower() in {"exit", "quit"}:
                break
            response = invoke_multiagent(user_input)
            print("🤖 Bot:", response)
            

2025-07-23 14:01:25.559 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 14:01:25.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 14:01:28.198 
  command:

    streamlit run C:\Users\Mahendra\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-23 14:01:28.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 14:01:28.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 14:01:28.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 14:01:28.209 Thread 'MainThread': missing ScriptRunContext! This warning ca